<img src="images/oasis-lmf-colour.png" alt="Oasis LMF logo" width="250" align="left"/>
<br><br><br>

# Exercise 1:   Exposure data in OED

The Oasis platform imports exposure data in OED format (Open Exposure Data). This is a completely open data format maintained by Modex. The full specification can be found [here](https://github.com/Simplitium/OED/). A subset of this information is given inline in the exercise. Later in the workshop we will be using the GEM Dominican Republic Earthquake model, so in this exercise we will construct some appropriate exposure data.

In [7]:
# Enable intellisense
%config IPCompleter.greedy=True

# General imports
import pandas as pd
import numpy as np
import os
import shutil
import jupyter_helper
jupyter_helper.set_style()

# Suppress warnings and logging that clutter the output
import logging
logger = logging.getLogger()
logger.disabled = True
import warnings
warnings.filterwarnings("ignore")

### Locations, accounts and policies

The set of location fields that we will use in this exercise are:

| Input Field Name | Type & Description | Required Field | Data Type   | Default |
|:---------------- | ------------------ | -------------- | ----------- | ------- |
| PortNumber	   | Portfolio number	| R	             | varchar(20) | n/a     |
| AccNumber	       | Account number	    | R	             | nvarchar(40)| n/a     |
| LocNumber	       | Location number	| R	             | nvarchar(20)| n/a     |
| AreaName	       | Name relating to the AreaCode (e.g. State name) | O | nvarchar(50) | n/a |
| GeogName1	       | Descriptor of the geographical area | O | nvarchar(50) | n/a |
| Latitude	       | Latitude in degrees (-90.0 to +90.0) |	O |	decimal	| 0 |
| Longitude	       | Longitude in degrees (-180.0 to +180.0) | O | decimal | 0 |
| CountryCode	   | Country code (based on ISO3166 alpha-2 codes) | R | char(2) | n/a |	
| BuildingTIV	   | Building Total Insured Value | R | float	| n/a |
| OccupancyCode	   | OED occupancy code	| O | int | 1000 |
| ConstructionCode | OED construction code | O |int | 5000 |
| NumberOfStoreys  | Number of storeys | O | tinyint | 0 |
| LocPerilsCovered | The perils covered by the location | R | varchar(250) | n/a |


A basic set of occupancy and construction codes are:

| OED Code | AIR Code |	Occupancy Name |
|:---------|----------|----------------|
| 1000     | 300	  | Unknown        |
| 1050	   | 301	  | Residential, General residential |
| 1100	   | 311	  | Commercial, General commercial   |
| 1150	   | 321	  | Industrial, General industrial   |

| OED Code | AIR Code |	Construction Name |
|:---------|----------|-------------------|
| 5000     | 100      | Unknown           |
| 5050     | 101      | Wood, Wood frame  |
| 5100     | 111      | Masonry, Masonry  |
| 5101     | 112      | Masonry, Adobe    |
| 5102     | 131      | Concrete, Reinforced concrete |

The set of account fields that we will use are:

| Input Field Name | Type & Description | Required Field | Data Type   | Default |
|:---------------- | ------------------ | -------------- | ----------- | ------- |
| PortNumber	   | Portfolio number	| R	             | varchar(20) | n/a |
| AccNumber	       | Account number	    | R	             | nvarchar(40)| n/a |
| AccPeril	       | Perils for account financial terms	| CR | varchar(250) |	n/a |
| PolNumber	       | Policy layer number | R             | varchar(20) | n/a |
| LayerNumber	   | Layer number	    | O | int | 1 | 
| LayerParticipation | Insurance company share of the policy layer (values between 0 and 1) | O | float | 1 |
| LayerLimit	   | Policy layer limit	| O | float | 0 |
| LayerAttachment | Policy layer attachment point | O | float | 0 |

Create two accounts with the following schedules. 

**The schedule data has started being filled out in the data-frames below, but with the occupancy and constructions classes set to unknown. Please complete the schedule for Account 2 by entering the correct occupancy and construction classes.**

### Account 1

Policy 1: 100% of 25000 xs 25000
Policy 2: 50% of 25000 xs 50000

| Municipality | Province | Latitude | Longitude | Construction | Occupancy | NumberOfStoreys | Building value |
|--|--|--|--|--|--|--|--|
| Pedro Brand | Santo Domingo | 18.64788 | -70.09339 | Wood frame | Residential | 1 | 100000 |
| Sabana Grande de Boyá | Monte Plata | 18.9582 | -69.85408 | Masonry | Residential | 3 | 25000 |
| Santa Cruz de Barahona | Barahona | 18.22769 | -71.1752 | Masonry | Residential | 1 | 27000 |

### Account 2

Policy 1: 100% of 25000 xs 25000
Policy 2: 50% of 25000 xs 50000

| Municipality | Province | Latitude | Longitude | Construction | Occupancy | NumberOfStoreys | Building value |
|--|--|--|--|--|--|--|--|
| Villa Vázquez | Monte Cristi | 19.8079 | -71.44 | Adobe | Commercial | 1 | 33000 |
| Guayubín | Monte Cristi | 19.73665 | -71.29112 | Adobe | Commercial | 1 | 38400 |
| Cabrera | María Trinidad Sánchez | 19.60861 | -69.97794 | Masonry |Commercial | 2 | 67200 |

In [8]:
locations_df = pd.DataFrame.from_dict({
    'PortNumber':           ['1', '1', '1', '1', '1', '1'],
    'AccNumber':            ['1', '1', '1', '2', '2', '2'],
    'LocNumber':            ['1', '2', '3', '4', '5', '6'],
    'GeogName':             ['Pedro Brand', 'Sabana Grande de Boyá', 'Santa Cruz de Barahona', 'Villa Vázquez', 'Guayubín', 'Cabrera'],
    'AreaName1':            ['Santa Domingo', 'Monte Plata', 'Barahona', 'Monte Cristi', 'Monte Cristi', 'María Trinidad Sánchez'],
    'Latitude':             [18.64788, 18.9582, 18.22769, 19.8079, 19.73665, 19.60861],
    'Longitude':            [-70.09339, -69.85408, -71.1752, -71.44, -71.29112, -69.97794],
    'CountryCode':          ['DR', 'DR', 'DR', 'DR', 'DR', 'DR'],
    'ConstructionCode':     [5000, 5000, 5100, 5000, 5000, 5000],
    'OccupancyCode':        [1000, 1000, 1000, 1000, 1000, 1000],
    'NumberOfStoreys':      [1, 3, 1, 1, 1, 2],
    'BuildingTIV':          [100000, 25000, 27000, 33000, 38400, 67300],
    'LocPerilsCovered':     ['QEQ', 'QEQ', 'QEQ', 'QEQ', 'QEQ', 'QEQ']
 })  
 
accounts_df = pd.DataFrame.from_dict({
    'PortNumber':           ['1',     '1',     '1',    '1'     ],
    'AccNumber':            ['1',     '1',     '2',    '2'     ],
    'PolNumber':            ['1',     '2',     '3',    '4'     ],
    'LayerAttachment':      [25000,   50000,   25000,  50000   ],
    'LayerLimit':           [25000,   25000,   25000,  25000   ],
    'LayerParticipation':   [1.0,     1.0,     1.0,    1.0     ]
})  

In [9]:
# Export the exposure data to file
src_dir = '/tmp/exercise_1_oed'
if os.path.exists(src_dir):
    shutil.rmtree(src_dir)
os.mkdir(src_dir)

run_dir = '/tmp/exercise_1_run'
if os.path.exists(run_dir):
    shutil.rmtree(run_dir)
os.mkdir(run_dir)

locations_df.to_csv(
    os.path.join(src_dir, "location.csv"),
    index=False
)
accounts_df.to_csv(
    os.path.join(src_dir, "account.csv"),
    index=False
)

In [10]:
# Run a deterministic analysis of the exposure files
! oasislmf exposure run -s /tmp/exercise_1_oed


Processing arguments

Running deterministic losses (GUL=True, IL=True, RIL=False)

STARTED: oasislmf.manager.__init__
COMPLETED: oasislmf.manager.__init__ in 0.0s
STARTED: oasislmf.manager.run_deterministic
STARTED: oasislmf.manager.generate_oasis_files
STARTED: oasislmf.model_preparation.gul_inputs.get_gul_input_items
COMPLETED: oasislmf.model_preparation.gul_inputs.get_gul_input_items in 0.17s
STARTED: oasislmf.model_preparation.gul_inputs.write_gul_input_files
STARTED: oasislmf.model_preparation.gul_inputs.write_items_file
STARTED: oasislmf.model_preparation.gul_inputs.write_coverages_file
COMPLETED: oasislmf.model_preparation.gul_inputs.write_coverages_file in 0.01s
COMPLETED: oasislmf.model_preparation.gul_inputs.write_items_file in 0.01s
COMPLETED: oasislmf.model_preparation.gul_inputs.write_gul_input_files in 0.06s
STARTED: oasislmf.model_preparation.summaries.get_summary_mapping
COMPLETED: oasislmf.model_preparation.summaries.get_summary_mapping in 0.0s
STARTED: oasislmf.model

In [11]:
# List out the available options
! oasislmf exposure run --help

usage: oasislmf exposure run [-h] [-V] [-C CONFIG] [-t TEST_CASE_NAME] -s
                             SRC_DIR [-r RUN_DIR] [-l LOSS_FACTOR]
                             [-a ALLOC_RULE] [-v] [-o OUTPUT_LEVEL]

Generates deterministic losses using the installed ktools framework given
    direct Oasis files (GUL + optionally IL and RI input files).

    The command line arguments can be supplied in the configuration file
    (``oasislmf.json`` by default or specified with the ``--config`` flag).
    Run ``oasislmf config --help`` for more information.

optional arguments:
  -h, --help            show this help message and exit
  -V, --verbose         Use verbose logging.
  -C CONFIG, --config CONFIG
                        MDK config. JSON file
  -t TEST_CASE_NAME, --test-case-name TEST_CASE_NAME
                        Test case name
  -s SRC_DIR, --src-dir SRC_DIR
                        Source files directory - should contain the OED
                        exposure file + optionally 

In particular, you can vary the loss percentage and output granularity. 

**Try entering 0.8 loss percentage and 'loc' for the output granularity and re-run the analysis.**

**Then try entering your own values and re-run the analysis.**

In [12]:
! oasislmf exposure run -s /tmp/exercise_1_oed -l 0.5 -o loc


Processing arguments

Running deterministic losses (GUL=True, IL=True, RIL=False)

STARTED: oasislmf.manager.__init__
COMPLETED: oasislmf.manager.__init__ in 0.0s
STARTED: oasislmf.manager.run_deterministic
STARTED: oasislmf.manager.generate_oasis_files
STARTED: oasislmf.model_preparation.gul_inputs.get_gul_input_items
COMPLETED: oasislmf.model_preparation.gul_inputs.get_gul_input_items in 0.19s
STARTED: oasislmf.model_preparation.gul_inputs.write_gul_input_files
STARTED: oasislmf.model_preparation.gul_inputs.write_items_file
STARTED: oasislmf.model_preparation.gul_inputs.write_coverages_file
COMPLETED: oasislmf.model_preparation.gul_inputs.write_coverages_file in 0.01s
COMPLETED: oasislmf.model_preparation.gul_inputs.write_items_file in 0.01s
COMPLETED: oasislmf.model_preparation.gul_inputs.write_gul_input_files in 0.07s
STARTED: oasislmf.model_preparation.summaries.get_summary_mapping
COMPLETED: oasislmf.model_preparation.summaries.get_summary_mapping in 0.0s
STARTED: oasislmf.model

### Reinsurance

Reinsurance is specified using two files, ri_info and ri_scope. Broadly, the former specifies the contract terms and the latter specifies what risks are covered. The set of ri_info fields that we will use in this exercise are:

| Input Field Name | Type & Description | Required Field | Data Type   | Default |
|:---------------- | ------------------ | -------------- | ----------- | ------- |
| ReinsNumber |	Reinsurance number | R | int | n/a |
| RiskLimit	| Reinsurance per-risk limit | O | float | 0 |
| RiskAttachment | Reinsurance per-risk attachment point | O | float | 0 |
| OccLimit | Reinsurance occurrence limit | O | float | 0 |
| OccAttachment	| Reinsurance per occurrence attachment	| O | float |YES | 0 |
| PlacedPercent	| Percentage of the limit that has been placed in the reinsurance market. | R |	float | n/a |
| InuringPriority |	Reinsurance inuring priority capturing the order in which reinsurance contracts apply. | R	| tinyint |	n/a |
| ReinsType	| Type of reinsurance contract | R | | varchar(3) | NO | n/a |

The set of ri_scope fields that we will use in this exercise are:

| Input Field Name | Type & Description | Required Field | Data Type   | Default |
|:---------------- | ------------------ | -------------- | ----------- | ------- |
| ReinsNumber | Reinsurance number to link with ReinsInfo file | R | int | n/a |
| PortNumber | Portfolio number: used for defining treaty scope in the reinsurance scope file | CR | varchar(20) | n/a |
| AccNumber | Account number: used for defining treaty scope in the reinsurance scope file	| CR | nvarchar(40)	| n/a |
| PolNumber | Policy number: used for defining treaty scope in the reinsurance scope file | CR | varchar(20)	| n/a |
| LocNumber	| Location number: used for defining treaty scope in the reinsurance scope file	| CR | nvarchar(20) | n/a |
| RiskLevel | Code to indicate the definition of 'risk' for the per-risk terms in this particular treaty | R | char(3) | n/a |







The data below creates a location level fac treaty that inures to the benefit of a cat treaty. 

**Try out some different options. Other treaty types can also be created, as described in the OED documentation.**

In [13]:
ri_info_df = pd.DataFrame.from_dict({
    "ReinsNumber":     [1,      2     ],
    'RiskLimit':       [1000,   0     ],
    'RiskAttachment':  [1000,   0     ],
    'OccLimit':        [0,      1000  ],
    'OccAttachment':   [0,      1000  ],
    'PlacedPercent':   [1.0,    1.0   ],
    'InuringPriority': [1,      2     ],
    'ReinsType':       ['FAC',  'CXL' ]})

ri_scope_df = pd.DataFrame.from_dict({
    'ReinsNumber': [1,     2  ],
    'PortNumber':  ['1',   '1'],
    'AccNumber':   ['1',   '' ],
    'PolNumber':   ['',    '' ],
    'LocNumber':   ['1',   '' ],
    'RiskLevel':   ['LOC', '' ]})
    
ri_info_df.to_csv(
    os.path.join(src_dir, "ri_info.csv"), index=False
)
ri_scope_df.to_csv(
    os.path.join(src_dir, "ri_scope.csv"), index=False
)

! oasislmf exposure run -s /tmp/exercise_1_oed -o acc -l 1.0


Processing arguments

Running deterministic losses (GUL=True, IL=True, RIL=True)

STARTED: oasislmf.manager.__init__
COMPLETED: oasislmf.manager.__init__ in 0.0s
STARTED: oasislmf.manager.run_deterministic
STARTED: oasislmf.manager.generate_oasis_files
STARTED: oasislmf.model_preparation.gul_inputs.get_gul_input_items
COMPLETED: oasislmf.model_preparation.gul_inputs.get_gul_input_items in 0.17s
STARTED: oasislmf.model_preparation.gul_inputs.write_gul_input_files
STARTED: oasislmf.model_preparation.gul_inputs.write_items_file
STARTED: oasislmf.model_preparation.gul_inputs.write_coverages_file
COMPLETED: oasislmf.model_preparation.gul_inputs.write_items_file in 0.03s
COMPLETED: oasislmf.model_preparation.gul_inputs.write_coverages_file in 0.0s
COMPLETED: oasislmf.model_preparation.gul_inputs.write_gul_input_files in 0.06s
STARTED: oasislmf.model_preparation.summaries.get_summary_mapping
COMPLETED: oasislmf.model_preparation.summaries.get_summary_mapping in 0.0s
STARTED: oasislmf.model_p

In [ ]:
# Run the FM directly via Python code.

# This can enable more complex analytical operations. In the example below, we 
# estimate the damage level (as a % of TIV) at which the portfolio attachs and exhausts.

from oasislmf.manager import OasisManager
om = OasisManager()

# At what level of loss do the accounts attach?
last_loss = 0
attachment = 0
exhaustion = 0
# Cycle through all percentage loss levels in 1% increments
for i in range (0, 100):
    # Generate the losses
    (gul_losses, il_losses, ri_losses) = om.run_deterministic(
        src_dir='/tmp/exercise_1_oed', 
        loss_percentage_of_tiv=i/100)
    total_loss = il_losses.loss.sum()
    
    # Has the portfolio attached?
    if attachment == 0 and total_loss > 0:
        attachment = i
    
    # Has the portfolio exhasuted?
    if i > 0 and total_loss > 0 and total_loss == last_loss:
        exhaustion = i
        break
        
    last_loss = total_loss
    
print("Attaching loss level: {}%; Exhausting loss level: {}%".format(attachment, exhaustion))
    